## Export All Data to Single CSV
This notebook will walk you through the process of exporting indicator data as a single CSV file

In [ ]:
import os
import pyodbc
import csv

### Test to see if you have the needed drivers installed to connect to an Access Database
Run the line below and if you see an emtpy array: `[]` as a result, you may need to install the 64-bit ACE drivers

*for reference https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access*

*for 64-bit drivers : https://www.microsoft.com/en-us/download/confirmation.aspx?id=13255*

In [ ]:
[x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]

### Set your working directory
example: *C:\users\me\working_directory\my_country*

In [ ]:
output_working_directory = r'C:\Users\adam6475\devinfo\tanzania'

### Enter the path to your DevInfo Access Database
example: *C:\users\me\working_directory\my_country\my_devinfo_database.mdb (.accdb)*

In [ ]:
access_database = r'C:\Users\adam6475\devinfo\tanzania\TSED_20180423.mdb'

### Enter the location and name for your output CSV file
example: *C:\users\me\working_directory\my_country\devinfo_output.csv*

In [ ]:
output_csv = r'C:\Users\adam6475\devinfo\tanzania\tz_out_new.csv'

## Execute the query against the DevInfo Access tables

In [ ]:
connStr = (
    r"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};"
    r"DBQ={};".format(access_database)
    )

cnxn = pyodbc.connect(connStr)

sql = """\
SELECT 
UT_Data.Indicator_NId AS INDICATOR_ID, 
UT_Indicator_en.Indicator_Name AS [INDICATOR], 
UT_Area_en.Area_ID AS REF_AREA_ID, 
UT_Area_en.Area_Name AS REF_AREA, 
UT_Data.Data_Value AS OBS_VALUE, 
UT_Unit_en.Unit_Name AS UNIT, 
UT_Subgroup_Vals_en.Subgroup_Val AS SUBGROUP, 
UT_Indicator_Classifications_en.IC_Name AS SOURCE, 
UT_Area_en.Area_Parent_NId AS REF_AREA_PARENT_ID, 
UT_TimePeriod.TimePeriod AS TIME_PERIOD

FROM 
UT_TimePeriod INNER JOIN (UT_Indicator_Classifications_en INNER JOIN ((((UT_Area_en INNER JOIN UT_Data ON UT_Area_en.Area_NId = UT_Data.Area_NId) LEFT JOIN UT_Indicator_en ON UT_Data.Indicator_NId = UT_Indicator_en.Indicator_NId) INNER JOIN UT_Subgroup_Vals_en ON UT_Data.Subgroup_Val_NId = UT_Subgroup_Vals_en.Subgroup_Val_NId) INNER JOIN UT_Unit_en ON UT_Data.Unit_NId = UT_Unit_en.Unit_NId) ON UT_Indicator_Classifications_en.IC_NId = UT_Data.Source_NId) ON UT_TimePeriod.TimePeriod_NId = UT_Data.TimePeriod_NId

ORDER BY UT_Data.Indicator_NId;
"""

crsr = cnxn.execute(sql)

rows = crsr.fetchall()

print ('sucessfully executed data query :: {} rows returned'.format(len(rows)))

## Write the result to a CSV file

In [ ]:
output_path = os.path.abspath(output_csv)

# set encoding 
# when reading some CSV files, there may be encoding issues that result in unicode characters appearing in the field names
# if you have any unexpected behavior that is related to this, either re-save your file with "utf-8" encoding
# or try using 'utf-8-sig' as your encoding value below
# Stack Overflow reference: https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string/17912811#17912811
encoding = 'utf-8'
with open(output_path, 'w', encoding=encoding, newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    header_row = [d[0] for d in crsr.description]
        
    writer.writerow(header_row)
    
    for row in rows:
        writer.writerow(row)

print ('csv successfully created at {}'.format(os.path.abspath(output_path)))